In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Посчитаем моменты в общем случае

$$
\int_{\gamma_1}^{\gamma2} p(x) f(x) d x 
$$

Введем замену переменной $x = b - t$.
Тогда задача сведется к следующей.

$$
\int_{\gamma_2 - b}^{\gamma_1 - b} p(b - t) f(b - t) d t = \int_{\gamma_2 - b}^{\gamma_1 - b} \tilde p(t) f(b - t) d t \approx \sum_{i=1}^{n} A_{i} f\left(b - t_{i}\right)
$$

где $\tilde p(t) =  t^{-\frac{1}{4}}$. Тогда

$$\mu_i = \int_{\gamma_2 - b}^{\gamma_1 - b} t^{-\frac{1}{4}} \cdot t^{i} dt = \frac{1}{i+1-\frac{1}{4}}x^{i+1-\frac{1}{4}} \bigg|^{b - \gamma_1}_{b - \gamma_2}$$


In [ ]:
def moment(gamma1, gamma2, k):
    """ Функция вычисления моментов """
    b = 3.2
    coef = k + 1 - 1/4
    return 1 / coef * ((b - gamma1) ** coef - (b - gamma2) ** coef)

In [ ]:
def f(x, a=1.7, b=3.2, alpha=0, beta=0.25):
    return 3 * np.cos(2 * x) * np.exp(2 * x / 3) + 5 * np.sin(2.5 * x) * np.exp(-x / 3) + 2 * x

In [ ]:
def newton_cotes(f, gamma1=1.7, gamma2=3.2, n_points=5,b=3.2, return_info=False):
    """ Построение квадратурной формулы Ньютона-Котеса """
    # Замена переменной
    x = np.linspace(gamma1, gamma2, n_points)
    t = b - x
    
    # Строим матрицу Вандермонда
    vander = np.vander(t, n_points, increasing=True).T
    
    # Находим моменты
    mu = np.array([moment(gamma1, gamma2, k) for k in range(len(x))])
    
    # Находим коэф в квадратурной форме,
    A = np.linalg.solve(vander, mu)
    
    if return_info is True:
        return {"mu": mu, "A": A, "S": sum(A * f(x))}
    
    return sum(A * f(x))

# Метод Гаусса

Рассмотрим полином $P_{m}$. Представим его в виде $P_{m}(x)=P_{m-n}(x) \omega(x)+P_{n-1}(x)$. Мы хотим, чтобы методическая погрешность была равна 0 для $m \le 2n-1$. Распишем методическую погрешность.

$$
R\left(P_{m}(x)\right)=\int_{a}^{b} p(x) P_{m-n}(x) \omega(x) d x-\sum_{i=1}^{n} A_{i} P_{m-n}\left(x_{i}\right) \omega\left(x_{i}\right)+R\left(P_{n-1}(x)\right)
$$

Если $w(x)$ выбрать ортогональным ко всем полиномам степени меньше n. Тогда первое слагаемое будет равно 0.
Построим $w(x) = x^{n}+a_{n-1} x^{n-1}+a_{n-2} x^{n-2}+\ldots+a_{0}$ из условия ортогональности 

$$
\int_{a}^{b} p(x)\left[x^{n}+a_{n-1} x^{n-1}+a_{n-2} x^{n-2}+\ldots+a_{0}\right] x^{s} d x=0, \quad 
s=\overline{0, n-1}
$$

Получаем систему линейных уравнений

$$
\sum_{i=0}^{n-1} a_{i} \mu_{s+i}=-\mu_{s+n}, \quad s=\overline{0, n-1}
$$
где $\mu_{i}$ $i$ый момент функции $p(x)$.

    Есть теорема, которая говорит о том, что корни w(x) при положительной весовой функции будут лежать внутри отрезка а, b. Но это условие может нарушаться из-за вычислительной погрешности. 
    
Поэтому если в качестве узлов мы возьмем корни $w$, то второе слагаемое тоже будет равно 0

Теперь у нас осталось третье слагаемое, которое зануляется с помощью условия равенства нулю методической погрешности для полиномов степени n - 1. Которое можно записать в данном виде:

$$
\sum_{i=0}^{n-1} A_{i} x_{i}^{s}=\mu_{s} ; \quad s=0,1, \ldots, n-1
$$

Коэффициенты в квадратурной формуле должны быть положительными, что в связи с вычислительной погрешностью может нарушаться. 

Окончательно интеграл вычисляется как:

$$
\int_{a}^{b} p(x) f(x) d x \approx \sum_{i=1}^{n} A_{i} f\left(x_{i}\right)
$$

In [ ]:
def gauss(f, gamma1=1.7, gamma2=3.2, b=3.2, n_points=10):
    """ Построение квадратурной формулы с помощью специального выбора узлов.
        Алгебраическа степень точности 2*n - 1, где n - число узлов """
    
    # Находим моменты
    moments = np.array([moment(gamma1, gamma2, k) for k in range(2*n_points)])
    
    # Решаем систему, чтобы найти коэф. узлового многочлена
    maxtix_of_moments = np.array([moments[s: s + n_points] for s in range(n_points)])
    poly_coeffs = np.linalg.solve(maxtix_of_moments, -moments[n_points:])
    
    # Строим полином w, находим узлы
    poly = np.poly1d([1] + list(reversed(poly_coeffs)))
    nodes = poly.roots
    
    # Из-за вычислительной погрешности, корни могут находиться вне интервала а, b 
    # выведем предупреждение, если это так
    if not ((gamma1 - b < - nodes).all() and (-nodes < 0).all()):
        print("Некоторые узлы выходят за промежуток интегрирования")
        
    # Находим коэф в квадратурной форме
    vander = np.vander(nodes, n_points, increasing=True).T
    quad_coeff = np.linalg.solve(vander, moments[:n_points])
    
    return sum(f(b - nodes) * quad_coeff)

In [ ]:
def quad(f, a=1.7, b=3.2, method="newton_cotes", n_split=10, n_points=3):
    """ Составная квадратурная формула """
    if method == "newton_cotes":
        method = newton_cotes
    if method == "gauss":
        method = gauss
    
    # Разбиваем на интервалы
    intervals = np.linspace(a, b, n_split)
    integral = 0
    for i in range(1, len(intervals)):
        
        # На каждом интервале применяем заданную квадратурную форму 
        integral += method(f, intervals[i - 1], intervals[i], n_points=n_points)
        
    return integral

In [ ]:
quad(f, method="gauss", n_split=2, n_points=3)

# Аналитическое обоснование процесса Эйткена   
  


Создаем 3 сетки с шагом $h$, $\frac{h}{L}$, $\frac{h}{L^2}$, то есть $h_1$, $h_2$, $h_3$ соответственно. Предполагаем, что для их методических погрешностях верно аcимптотическое разложение

$$
R_{h_{i}}=J(f)-S_{h_{i}}=C_{m} h_{i}^{m}+O\left(h_{i}^{m+1}\right)
$$

Выражая $J(f)$ и приравнивания, получаем

$$
S_{h_{2}}-S_{h_{1}}=C_{m} h^{m}\left(1-\frac{1}{L^{m}}\right)+O\left(h^{m+1}\right)
$$

$$
S_{h_{3}}-S_{h_{2}}=C_{m} \left(\frac{h}{L}\right)^{m}\left(1-\frac{1}{L^{m}}\right)+O\left(h^{m+1}\right)
$$

Из этого следует

$$
\frac{S_{h_{3}}-S_{h_{2}}}{S_{h_{2}}-S_{h_{1}}} \approx \frac{C_{m}\left(\frac{h}{L}\right)^{m}\left(1-\frac{1}{L^{m}}\right)}{C_{m} h^{m}\left(1-\frac{1}{L^{m}}\right)}=\frac{1}{L^{m}}
$$

и

$$
m \approx-\frac{\ln \frac{S_{h_{3}}-S_{h_{2}}}{S_{h_{2}}-S_{h_{1}}}}{\ln L}
$$


In [ ]:
def aitken_process(f, L = 2, h = 0.2, a=1.7, b=3.2, return_sums=False, 
                   method="newton_cotes", n_points=3):
    
    assert type(L) == int, "L should be integer"
    n_split = int((b - a) / h)
    
    # Вычисляем квадратурную форму на трех сетках
    first_sum = quad(f, method=method, n_split=n_split, n_points=n_points)
    second_sum = quad(f, method=method, n_split=n_split * L, n_points=n_points)
    fird_sum = quad(f, method=method, n_split=n_split * L ** 2, n_points=n_points)
    
    # Находим приближенное значение m
    m = - np.log(abs((fird_sum - second_sum) / (second_sum - first_sum))) / np.log(L)
    
    if return_sums:
        return m, first_sum, second_sum, fird_sum
    
    return m

# Правило Рунге  
После получения значения m, мы можем оценить методическую погрешность каждой из сеток.
Из 

$$
S_{h_{2}}-S_{h_{1}}=C_{m} h^{m}\left(1-\frac{1}{L^{m}}\right)+O\left(h^{m+1}\right)
$$

следует 

$$
C_{m}=\frac{S_{h_{2}}-S_{h_{1}}}{h^{m}\left(1-L^{-m}\right)}+O\left(h^{m+1}\right)
$$

И подставляя значение $С_m$ в представление асимптотической поргешности для $h_1$, $h_2$ получаем

$$
R_{h_{1}}=J(f)-S_{h_{1}} \approx \frac{S_{h_{2}}-S_{h_{1}}}{1-L^{-m}}
$$

$$
R_{h_{2}}=J(f)-S_{h_{2}} \approx \frac{S_{h_{2}}-S_{h_{1}}}{L^{m}-1}
$$

Причем мы хотим найти такое $h_{opt}$, чтобы методическая погрешность была равна наперед заданному числу $\varepsilon$

$$
\varepsilon=\left|R_{h_{o p t}}\right| \approx\left|C_{m} h_{o p t}^{m}\right| \approx \frac{\left|S_{h_{2}}-S_{h_{1}}\right|}{h^{m}\left(1-L^{-m}\right)} h_{o p t}^{m}=R_{h_{1}}\left(\frac{h_{o p t}}{h}\right)^{m}
$$

Отсюда получаем приблизительный оптимальный шаг разбиения

$$
h_{o p t}=h\left(\frac{\varepsilon\left(1-L^{-m}\right)}{\left|S_{h_{2}}-S_{h_{1}}\right|}\right)^{\frac{1}{m}}=h_{1} \sqrt[m]{\frac{\varepsilon}{\left|R_{h_{1}}\right|}}=h_{2} \sqrt[m]{\frac{\varepsilon}{\left|R_{h_{2}}\right|}}
$$

In [68]:
def aitken_runge_process(f, L=2, h=0.2, a=1.7, b=3.2, epsilon=1e-6, 
                         method="newton_cotes", n_points=3, return_info=True):
    inaccuracy = epsilon + 1

    # Пока методическая погрешность больше эпсилон, находим новые разбиения
    while abs(inaccuracy) > epsilon:
        # Находим m и все S_h_i
        m, first_sum, second_sum, fird_sum = aitken_process(f, L=L, h=h, a=a, b=b, return_sums=True, 
                                                            method=method, n_points=n_points)
        
        # Находим приблизительную методическую погрешность
        inaccuracy = (second_sum - first_sum) / (L ** m - 1)
        
        if return_info:
            C = (second_sum - first_sum) / (h ** m) / (1 - L ** (-m))

        # Находим оптимальное h
        h = h / L * (epsilon / abs(inaccuracy)) ** (1 / m)
    
    if return_info:
        return second_sum, inaccuracy, m, C
        
    return second_sum, inaccuracy

In [71]:
from scipy.integrate import quad as best_quad

def F(x, a=1.7, b=3.2, alpha=0, beta=0.25):
    f = 3 * np.cos(2 * x) * np.exp(2 * x / 3) + 5 * np.sin(2.5 * x) * np.exp(-x / 3) + 2 * x
    p = (x - a) ** alpha * (b - x) ** beta
    return f / p

integral = best_quad(F, a=1.7, b=3.2, epsabs=1e-10)[0]

print("Составная квадратурная формула на основе трех точечного Ньютона-Котеса")

for epsilon in [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
    my_integral, inaccuracy, m, C = aitken_runge_process(f, method="newton_cotes",
                                                           epsilon=epsilon, return_info=True)
    dif = abs(my_integral - integral)
    print(f"Разница между моим значением и эталонным при {epsilon = } = {dif}")
    print(f"Значение {m = }, {C = }", "\n")

Составная квадратурная формула на основе трех точечного Ньютона-Котеса
Разница между моим значением и эталонным при epsilon = 0.0001 = 5.370202320165163e-06
Значение m = 3.904171702258429, C = 0.667088232239517 

Разница между моим значением и эталонным при epsilon = 1e-05 = 6.594037742502223e-07
Значение m = 3.8243577848038157, C = 0.5409487352595536 

Разница между моим значением и эталонным при epsilon = 1e-06 = 7.652460709550724e-08
Значение m = 3.780298045764965, C = 0.43918138995035066 

Разница между моим значением и эталонным при epsilon = 1e-07 = 9.580215731830322e-09
Значение m = 3.757902186778894, C = 0.4050528371058341 

Разница между моим значением и эталонным при epsilon = 1e-08 = 1.2098233526103286e-09
Значение m = 3.746475755101362, C = 0.3892540080054902 

Разница между моим значением и эталонным при epsilon = 1e-09 = 1.4934187220205786e-10
Значение m = 3.7408519590078813, C = 0.37421015808679686 

Разница между моим значением и эталонным при epsilon = 1e-10 = 1.806199

In [78]:
print("Составная квадратурная формула на основе двух точечного Гаусса")
for epsilon in [1e-4, 1e-5, 1e-6, 1e-7, 1e-8]:
    my_integral, inaccuracy, m, C = aitken_runge_process(f, method="newton_cotes", n_points=2,
                                                           epsilon=epsilon, return_info=True)
    dif = abs(my_integral - integral)
    print(f"Разница между моим значением и эталонным при {epsilon = } = {dif}")
    print(f"Значение {m = }, {C = }", "\n")

Составная квадратурная формула на основе двух точечного Гаусса
Разница между моим значением и эталонным при epsilon = 0.0001 = 1.1469997268420684e-05
Значение m = 1.992994903581896, C = 0.8804277153530117 

Разница между моим значением и эталонным при epsilon = 1e-05 = 8.242169577954428e-07
Значение m = 1.996714780065881, C = 0.8932902678353573 

Разница между моим значением и эталонным при epsilon = 1e-06 = 5.936375302439956e-08
Значение m = 1.9985730733073566, C = 0.9034777957840575 

Разница между моим значением и эталонным при epsilon = 1e-07 = 4.277410425856942e-09
Значение m = 1.9994351159610328, C = 0.9100552858132505 

Разница между моим значением и эталонным при epsilon = 1e-08 = 3.076294774473354e-10
Значение m = 1.999900165270183, C = 0.9142159479779176 



Видно, что значения m, Cm на кажой итерации изменяются все меньше и меньше, значит мы приближаемся к области асимптотики. С другой стороны, может быть, что мы уже там находится и изменения в m, С вызваны вычислительной погрешностью. 